In [68]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE

In [69]:
# loading in London weather dataset
df = pd.read_csv("/Users/cartermain/Downloads/london_weather.csv")

In [70]:
# counting null values
print(df.isnull().sum())
print("\n")

date                   0
cloud_cover           19
sunshine               0
global_radiation      19
max_temp               6
mean_temp             36
min_temp               2
precipitation          6
pressure               4
snow_depth          1441
dtype: int64




Going to need to replace null values for snow depth with 0. Let's start with days that had 0 recorded precipitation.

In [71]:
# counting rows that meet this criteria
print(sum((df["snow_depth"].isnull()) & (df["precipitation"] == 0)))

758


In [72]:
# replacing all null snow_depth days that had 0 precipitation with 0 for snow_depth
df.loc[(df["snow_depth"].isnull()) & (df["precipitation"] == 0), "snow_depth"] = 0

Now, we can replace all days with a min temp above the temperature needed for snow to accumulate on the ground (5° C according to the National Snow & Ice Data Center: https://nsidc.org/learn/parts-cryosphere/snow/science-snow)

In [73]:
# identifying number of rows that meet this criteria
print(sum((df["snow_depth"].isnull()) & (df["min_temp"] > 5)))

625


In [74]:
# replacing null values with 0 for snow_depth
df.loc[(df["snow_depth"].isnull()) & (df["min_temp"] > 5), "snow_depth"] = 0

In [83]:
# taking a look at fresh null count after changes
print(df.isnull().sum())
print(len(df))

date                 0
cloud_cover         19
sunshine             0
global_radiation    19
max_temp             6
mean_temp           36
min_temp             2
precipitation        6
pressure             4
snow_depth          58
dtype: int64
15341


We've now gotten null value count down to less than 1% of the original dataset size so I am comfortable dropping remaining rows with null values

In [85]:
# dropping rows with null values
df.dropna(inplace = True)

Now, let's check for some outliers before getting into our model

In [87]:
# checking max and min values relative to mean, 25%, and 75%
print(df.describe())

               date   cloud_cover      sunshine  global_radiation  \
count  1.520700e+04  15207.000000  15207.000000      15207.000000   
mean   1.999494e+07      5.269678      4.349464        119.022555   
std    1.209193e+05      2.067057      4.022147         88.942076   
min    1.979010e+07      0.000000      0.000000          8.000000   
25%    1.989061e+07      4.000000      0.500000         41.000000   
50%    1.999112e+07      6.000000      3.500000         95.000000   
75%    2.010061e+07      7.000000      7.200000        186.000000   
max    2.020123e+07      9.000000     16.000000        402.000000   

           max_temp     mean_temp      min_temp  precipitation       pressure  \
count  15207.000000  15207.000000  15207.000000   15207.000000   15207.000000   
mean      15.402223     11.493293      7.576511       1.666739  101536.427303   
std        6.549930      5.723116      5.323478       3.736406    1048.041358   
min       -6.200000     -7.600000    -11.800000       

In [89]:
# creating a list of columns to check for outliers in (ignoring snow depth becaude 25% and 75% are 0)
columns_to_inspect = ["cloud_cover", "sunshine", "global_radiation", "max_temp", "mean_temp", "min_temp", "precipitation", "pressure"]

In [93]:
# using interquartile range to turn outliers into null values
for feature in columns_to_inspect:
    # looking at top and bottom 100 values for each feature to expedite the loop and save space
    for x in df[feature].sort_values(ascending = False)[0:100]:
        q75 = df[feature].describe()["75%"]
        q25 = df[feature].describe()["25%"]
        intr_qr = q75 - q25
        max = q75 + (1.5 * intr_qr)
        min = q25 - (1.5 * intr_qr)
        if x > max or x < min:
            df.loc[df[feature] == x, feature] = np.nan
    for x in df[feature][df[feature].notnull() == True].sort_values(ascending = False)[-100:]:
        if x > max or x < min:
            df.loc[df[feature] == x, feature] = np.nan

In [94]:
# checking null count after loop
print(df.isnull().sum())

date                  0
cloud_cover           0
sunshine              0
global_radiation      0
max_temp             11
mean_temp             1
min_temp              7
precipitation       408
pressure            293
snow_depth            0
dtype: int64


In [95]:
# dropping na values
df.dropna(inplace = True)

In [96]:
# checking how these drops changed each feature
print(df.describe())

               date   cloud_cover     sunshine  global_radiation  \
count  1.450900e+04  14509.000000  14509.00000      14509.000000   
mean   1.999463e+07      5.224895      4.45693        121.255014   
std    1.207134e+05      2.074654      4.04366         89.389908   
min    1.979010e+07      0.000000      0.00000         10.000000   
25%    1.989061e+07      4.000000      0.50000         43.000000   
50%    1.999103e+07      6.000000      3.70000         99.000000   
75%    2.010052e+07      7.000000      7.30000        190.000000   
max    2.020123e+07      9.000000     16.00000        402.000000   

           max_temp     mean_temp      min_temp  precipitation       pressure  \
count  14509.000000  14509.000000  14509.000000   14509.000000   14509.000000   
mean      15.520367     11.549631      7.590950       1.177304  101612.707285   
std        6.548122      5.729887      5.326527       2.262090     961.825556   
min       -4.000000     -5.400000     -8.500000       0.000000 

Now that we have gotten rid of outliers, we can run our linear regression with scaled data to predict pressure

In [98]:
# collecting features
features = df[["date", "pressure", "cloud_cover", "global_radiation", "max_temp", "mean_temp", "min_temp", "precipitation", "snow_depth"]]

In [21]:
# scaling feature set
scaler = StandardScaler()
scaler.fit_transform(features)

array([[-1.69742021,  0.33547089, -1.57552003, ..., -2.84262071,
        -0.36037083, 18.59207728],
       [-1.69741192,  0.96137317,  0.3588995 , ..., -2.84262071,
        -0.49426323, 16.51908968],
       [-1.69740364,  0.48449524, -0.12470538, ..., -2.78609674,
        -0.49426323,  8.22713926],
       ...,
       [ 1.70822017, -2.71456088,  0.84250438, ..., -1.22226685,
        -0.49426323, -0.06481116],
       [ 1.70822845, -1.35347178,  0.3588995 , ..., -1.44836274,
        -0.49426323, -0.06481116],
       [ 1.70823673, -1.05542308,  0.84250438, ..., -2.01360246,
        -0.49426323, -0.06481116]])

In [99]:
# train test splitting with random state of 42
x_train, x_test, y_train, y_test = train_test_split(features, df["sunshine"], train_size = 0.7, random_state = 42)

In [100]:
# training and scoring model
model = LinearRegression()
model.fit(x_train, y_train)
print(model.score(x_test, y_test))

0.8762240473059097


Time to run RFE to see which features we can drop

In [115]:
# running via a for loop to test every possible number of features and only keep if it resulted in a 10%+ increase to save computational space
max_score = 0.01
best_x = 0
for x in range(1,len(features.columns) + 1):
    rfe = RFE(estimator = model, n_features_to_select = x)
    rfe.fit(x_train, y_train)
    score = rfe.score(x_test, y_test)
    if (score - max_score) / max_score > .10:
        max_score = score
        best_x = x
print(max_score, best_x)

0.8751743916548861 6


In [116]:
# finding out which features RFE decided to keep and drop
rfe = RFE(estimator = model, n_features_to_select = best_x)
rfe.fit(x_train, y_train)
rfe.score(x_test, y_test)
print(rfe.support_)

[False False  True  True  True  True  True  True False]


In [117]:
# collecting column names of features kept by RFE
kept_features = []
y = 0
for kept in rfe.support_:
    if kept == True:
        kept_features.append(features.columns[y])
    y += 1

In [105]:
# creating revised feature set
features_2 = df[kept_features]

In [106]:
# scaling new features set 
scaler_2 = StandardScaler()
scaler_2.fit_transform(features_2)

array([[-1.55447909, -0.77477875, -2.01902502, -2.73132254, -2.83326642,
        -0.34363378],
       [ 0.37361983, -1.05446202, -2.12592958, -2.46952827, -2.83326642,
        -0.5204675 ],
       [-0.1084049 , -1.21108465, -2.17174582, -2.50443418, -2.77694261,
        -0.5204675 ],
       ...,
       [ 0.37361983, -0.90902672, -1.66776717, -1.5619748 , -1.29374877,
        -0.5204675 ],
       [ 0.37361983, -1.11039867, -1.51504637, -1.54452185, -1.44394561,
        -0.5204675 ],
       [ 0.85564456, -0.9761507 , -2.14120166, -2.15537515, -2.00718378,
        -0.5204675 ]])

In [108]:
# resplitting with new feature set keeping random state as 42
x_train_2, x_test_2, y_train_2, y_test_2 = train_test_split(features_2, df["sunshine"], train_size = 0.7, random_state = 42)

In [109]:
# retraining model with revised feature set
model.fit(x_train_2, y_train_2)
print(model.score(x_test_2, y_test_2))

0.8751743916548861


In [110]:
# running a for loop to test train sizes in 0.05 increments and find the top performing train size
best_split = 0
best_score = 0
for x in np.linspace(0, 1, 21):
    if x == 0 or x == 1:
        continue
    else: 
        x_train_3, x_test_3, y_train_3, y_test_3 = train_test_split(features_2, df["sunshine"], train_size = x, random_state = 42)
        model.fit(x_train_3, y_train_3)
        score = model.score(x_test_3, y_test_3)
        if score > best_score:
            best_score = score
            best_split = x

In [111]:
# finding best performers
print(best_score, best_split)

0.8841681084240293 0.8500000000000001


In [113]:
# resplitting revised feature set at best train size
best_x_train, best_x_test, best_y_train, best_y_test = train_test_split(features_2, df["sunshine"], train_size = best_split, random_state = 42)

In [114]:
# retraining our model one final time with this new split
model.fit(best_x_train, best_y_train)
print(model.score(best_x_test, best_y_test))

0.8841681084240293


Lastly, let's check the coefficients of each feature for our model

In [118]:
# running a for loop to print feature name and rounded coefficient
x = 0
for feature in features_2.columns:
    print(feature + ": " + str(round(model.coef_[x],2)))
    x += 1

cloud_cover: -0.74
global_radiation: 0.03
max_temp: -0.09
mean_temp: 0.15
min_temp: -0.14
precipitation: -0.05
